# Variables status (private or shared)

---
**Requirements:**

- [Get started](./Get_started.ipynb)

---

## Default status of scalar and arrays

The default status of variables depend on what they are (scalar or array) and the compute construct you use.
Here is a summary:

|              | Scalar              | Array     |
|--------------|---------------------|-----------|
| _parallel_   | _gang_ firstprivate | shared    |
| _kernels_    | shared              | shared    |

## Private variables

It is possible to make a variable private at _gang_, _worker_ or _vector_ level of parallelism if you use the `acc loop` clauses _private_ or _firstprivate_.
The variables will be private at the maximum level of parallelism the loop works.

Here are some examples:

### Simple cases
A single loop with variables private at gang level:

```fortran
real(kind=8) :: scalar
real(kind=8), dimension(:) :: array
!$acc parallel
    !$acc loop gang private(scalar, array)
    do i=0, array_size
        !! do some work on scalar and array
    enddo
!$acc end parallel
```

A single loop with variables private at worker level:

```fortran
real(kind=8) :: scalar
real(kind=8), dimension(:) :: array
!$acc parallel
    !$acc loop gang worker private(scalar, array)
    do i=0, array_size
        !! do some work on scalar and array
    enddo
!$acc end parallel
```

A single loop with variables private at vector level:

```fortran
real(kind=8) :: scalar
real(kind=8), dimension(:) :: array
!$acc parallel
    !$acc loop gang vector private(scalar, array)
    do i=0, array_size
        !! do some work on scalar and array
    enddo
!$acc end parallel
```

### A bit less straightforward
Nested loops:

```fortran
real(kind=8) :: scalar1
real(kind=8) :: scalar2

scalar1 = 0
!$acc parallel
    !$acc loop gang reduction(+:scalar1) private(scalar2)
    do i=1, size_i
        scalar2 = 0
        !! scalar2 is private at gang level but shared at worker/vector level
        !$acc loop vector reduction(+:scalar2)
        do j=1, size_j
            scalar2 = scalar2 + ...
        enddo
        scalar1 = scalar1 + scalar2
    enddo
!$acc end parallel
```

## Caution

You can make arrays private but in this case the memory requirements might be huge
if you want them to be private at _worker_ or _vector_ level.